# Recurrent Network

In this Notebook, we will do over Recurrent Networks

In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
# import dataset, network to train and metric to optimize
from pytorch_forecasting import TimeSeriesDataSet, RecurrentNetwork, QuantileLoss
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
# pytorch_forecasting.metrics.RMSE

In [3]:
import pandas as pd
df = pd.read_csv("scpppi/processed_data_l.csv")

In [4]:

def clean_df(df):
    df = df.drop(columns=["Unnamed: 0","datetime","granule_id"])
    df["datetime_dt"] = pd.to_datetime(df["datetime_dt"] )
    df["month"] = [i.month for i in df["datetime_dt"]]
    df["month"] = df["month"].astype(str).astype("category")
    one_hot = pd.get_dummies(df["month"])
    df = df.join(one_hot)
    df = df.drop('month',axis = 1)
    df = df.drop('date',axis = 1)
#     one_hot = pd.get_dummies(df["location"])
#     df = df.join(one_hot)
#     one_hot = pd.get_dummies(df["grid_id"])
#     df = df.join(one_hot)
#     df = df.drop('grid_id',axis = 1)
    return df

In [5]:
df_cleaned = clean_df(df)

In [6]:
df_cleaned.head()

,grid_id,value,datetime_dt,location,AOD at 0.47 micron_0,AOD at 0.47 micron_1,AOD at 0.47 micron_2,AOD at 0.47 micron_3,AOD at 0.47 micron_4,AOD at 0.47 micron_5,...,11,12,2,3,4,5,6,7,8,9
0,3S31A,11.4,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
1,A2FBI,17.0,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
2,DJN0F,11.1,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
3,E5P9N,22.1,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0
4,FRITQ,29.8,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,0,1,0,0,0,0,0,0,0


In [7]:
data = df_cleaned

In [8]:
data["time_idx"] = data["datetime_dt"].dt.year * 365 + data["datetime_dt"].dt.month *30 + data["datetime_dt"].dt.day
data["time_idx"] -= data["time_idx"].min()

In [36]:
data = data.sort_values("time_idx")

In [37]:
max_prediction_length = 100
max_encoder_length = 18
training_cutoff = data["time_idx"].max() - max_prediction_length


In [38]:
time_varying_known_reals = [i.replace(".","_") for i in list(data.columns[4:])]
time_varying_known_reals[0:2]

['AOD at 0_47 micron_0', 'AOD at 0_47 micron_1']

In [39]:
time_varying_known_reals

['AOD at 0_47 micron_0',
 'AOD at 0_47 micron_1',
 'AOD at 0_47 micron_2',
 'AOD at 0_47 micron_3',
 'AOD at 0_47 micron_4',
 'AOD at 0_47 micron_5',
 'AOD at 0_47 micron_6',
 'AOD at 0_47 micron_7',
 'AOD at 0_47 micron_8',
 'AOD at 0_47 micron_9',
 'AOD at 0_47 micron_10',
 'AOD at 0_47 micron_11',
 'AOD at 0_47 micron_12',
 'AOD at 0_47 micron_13',
 'AOD at 0_47 micron_14',
 'AOD at 0_47 micron_15',
 'AOD at 0_47 micron_16',
 'AOD at 0_47 micron_17',
 'AOD at 0_47 micron_18',
 'AOD at 0_47 micron_19',
 'AOD at 0_47 micron_20',
 'AOD at 0_47 micron_21',
 'AOD at 0_47 micron_22',
 'AOD at 0_47 micron_23',
 'AOD at 0_47 micron_24',
 'AOD at 0_47 micron_25',
 'AOD at 0_47 micron_26',
 'AOD at 0_47 micron_27',
 'AOD at 0_47 micron_28',
 'AOD at 0_47 micron_29',
 'AOD at 0_47 micron_30',
 'AOD at 0_47 micron_31',
 'AOD at 0_47 micron_32',
 'AOD at 0_47 micron_33',
 'AOD at 0_47 micron_34',
 'AOD at 0_47 micron_35',
 'AOD at 0_47 micron_36',
 'AOD at 0_47 micron_37',
 'AOD at 0_47 micron_3

In [40]:
data.columns = list(data.columns)[:4]+ time_varying_known_reals

In [41]:
data


,grid_id,value,datetime_dt,location,AOD at 0_47 micron_0,AOD at 0_47 micron_1,AOD at 0_47 micron_2,AOD at 0_47 micron_3,AOD at 0_47 micron_4,AOD at 0_47 micron_5,...,12,2,3,4,5,6,7,8,9,time_idx
0,3S31A,11.400000,2018-02-01 08:00:00+00:00,Los Angeles (SoCAB),-0.563344,0.109907,-0.103034,0.234161,-0.041027,0.229606,...,0,1,0,0,0,0,0,0,0,0
21430,P8JA5,146.370370,2018-02-01 18:30:00+00:00,Delhi,-0.551273,0.114632,-0.116517,0.256812,-0.049742,0.242454,...,0,1,0,0,0,0,0,0,0,0
21429,KZ9W9,101.304348,2018-02-01 18:30:00+00:00,Delhi,-0.551273,0.114632,-0.116517,0.256812,-0.049742,0.242454,...,0,1,0,0,0,0,0,0,0,0
21428,HM74A,154.875000,2018-02-01 18:30:00+00:00,Delhi,-0.551273,0.114632,-0.116517,0.256812,-0.049742,0.242454,...,0,1,0,0,0,0,0,0,0,0
21427,HANW9,148.687500,2018-02-01 18:30:00+00:00,Delhi,-0.551273,0.114632,-0.116517,0.256812,-0.049742,0.242454,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21410,VR4WG,16.870968,2020-12-31 16:00:00+00:00,Taipei,-0.550836,0.114607,-0.116815,0.256702,-0.049746,0.242348,...,1,0,0,0,0,0,0,0,0,1060
21411,XJF9O,12.564516,2020-12-31 16:00:00+00:00,Taipei,-0.550836,0.114607,-0.116815,0.256702,-0.049746,0.242348,...,1,0,0,0,0,0,0,0,0,1060
21412,XNLVD,12.500000,2020-12-31 16:00:00+00:00,Taipei,-0.550836,0.114607,-0.116815,0.256702,-0.049746,0.242348,...,1,0,0,0,0,0,0,0,0,1060
37204,ZF3ZW,410.500000,2020-12-31 18:30:00+00:00,Delhi,-0.550824,0.114607,-0.116819,0.256715,-0.049741,0.242339,...,1,0,0,0,0,0,0,0,0,1060


In [42]:
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="value",
    group_ids=["location","grid_id"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=["location","grid_id"],
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "value"
    ],
    target_normalizer=GroupNormalizer(
        groups=[ "location","grid_id"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

In [43]:
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=16)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=16)

Exception ignored in: <function _ConnectionBase.__del__ at 0x7f00f81bb940>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 132, in __del__
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
    self._close()
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1238: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dat

In [44]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)


78.26200866699219

In [45]:
from torch.nn import MSELoss

In [46]:

from torchmetrics import MeanSquaredError


In [53]:
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=10,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [54]:


tft = RecurrentNetwork.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=8,  # most important hyperparameter apart from learning rate
    rnn_layers= 2,
    dropout=0.3,  # between 0.1 and 0.3 are good values
    output_size=1,  
    loss=RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Number of parameters in network: 68.4k


In [55]:
import torch.nn.functional as F

In [57]:
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type           | Params
----------------------------------------------------
0 | loss             | RMSE           | 0     
1 | logging_metrics  | ModuleList     | 0     
2 | embeddings       | MultiEmbedding | 819   
3 | rnn              | LSTM           | 67.6 K
4 | output_projector | Linear         | 9     
----------------------------------------------------
68.4 K    Trainable params
0         Non-trainable params
68.4 K    Total params
0.274     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
Global seed set to 42


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/sit

...

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/sit

Validating: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/sit

Validating: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)


In [ ]:
index 1 is out of bounds

In [19]:
for i,j in enumerate(train_dataloader):
    print(i,j)
    break

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
/opt/conda/lib/python3.8/sit

0 ({'encoder_cat': tensor([[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]]]), 'encoder_cont': tensor([[[ 8.8889, -1.0000,  0.0000,  ..., -1.7260, -4.9444, -0.1616],
         [ 8.8889, -1.0000,  0.0000,  ..., -1.7260, -4.8889,  0.5079],
         [ 8.8889, -1.0000,  0.0000,  ..., -1.7260, -4.8333, -0.1974],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 

In [20]:
j[0].keys()

dict_keys(['encoder_cat', 'encoder_cont', 'encoder_target', 'encoder_lengths', 'decoder_cat', 'decoder_cont', 'decoder_target', 'decoder_lengths', 'decoder_time_idx', 'groups', 'target_scale'])

In [61]:
trainer.predict(model = tft, dataloaders=val_dataloader,return_predictions=True)




TypeError: predict() got an unexpected keyword argument 'mode'

In [64]:
for i,j in enumerate(val_dataloader):
    print(i,j)#.data[0]
    break

/opt/conda/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)


0 ({'encoder_cat': tensor([[[ 0,  1],
         [ 0,  1],
         [ 0,  1],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]],

        [[ 0,  3],
         [ 0,  3],
         [ 0,  3],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]],

        [[ 0,  4],
         [ 0,  4],
         [ 0,  4],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]],

        ...,

        [[ 2, 42],
         [ 2, 42],
         [ 2, 42],
         ...,
         [ 2, 42],
         [ 2, 42],
         [ 2, 42]],

        [[ 2, 48],
         [ 2, 48],
         [ 2, 48],
         ...,
         [ 2, 48],
         [ 2, 48],
         [ 2, 48]],

        [[ 2, 49],
         [ 2, 49],
         [ 2, 49],
         ...,
         [ 2, 49],
         [ 2, 49],
         [ 2, 49]]]), 'encoder_cont': tensor([[[ 1.1111,  0.9645,  0.9599,  ...,  1.3726, -1.0556, -0.6921],
         [ 1.1111,  0.9645,  0.9599,  ...,  1.3765, -1.0000, -0.6904],
         [ 1.1111,  0.9645, 

In [73]:
res = tft.cpu().forward(j[0])

In [95]:
for i,j in res.items():
    print(j.detach().numpy().shape)

(53, 100, 1)


In [85]:
len(set(data["grid_id"]))

54

In [91]:
len(set( data[lambda x: x.time_idx >= training_cutoff]["grid_id"]))

49

In [109]:
j[1][0].shape

torch.Size([64, 1])

In [104]:
j[0]["groups"]

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])